In [0]:
from datetime import datetime
import requests
import os
from pyspark.sql import SparkSession

In [0]:
# Cria sessão Spark
spark = SparkSession.builder.getOrCreate()

In [0]:
# =====================================
# PARÂMETROS DO JOB (Widgets)
# =====================================
dbutils.widgets.text("data_ref", "", "Data de Referência (AAAAMMDD)")
dbutils.widgets.text("url_csv", "", "URL do CSV (opcional)")
dbutils.widgets.text("destino", "", "Destino no Volume (ex: /Volumes/datamasters/raw/raw_inpe)")

In [0]:
# Lendo parâmetros
data_ref = dbutils.widgets.get("data_ref")
url_csv = dbutils.widgets.get("url_csv")
destino_base = dbutils.widgets.get("destino")

In [0]:
# =====================================
# VALIDAÇÃO E DEFAULTS
# =====================================
# Se não for passada, usa a data do dia
if not data_ref:
    data_ref = datetime.now().strftime("%Y%m%d")

# Se não for passada, monta a URL padrão
if not url_csv:
    url_csv = f"https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/diario/Brasil/focos_diario_br_{data_ref}.csv"
else:
    # Remove possíveis sufixos e barras finais
    url_csv = url_csv.rstrip('/').removesuffix('.csv')
    url_csv = f"{url_csv}{data_ref}.csv"
# Se não for passado destino, define o padrão
if not destino_base:
    destino_base = "/Volumes/datamasters/raw/raw_inpe"

# Caminho final
destino_final = f"{destino_base}/focos_diario_br_{data_ref}.csv"

In [0]:
# =====================================
# LOG
# =====================================
print("=====================================")
print(f"Data de referência : {data_ref}")
print(f"URL de origem      : {url_csv}")
print(f"Destino final      : {destino_final}")
print("=====================================")

In [0]:
# =====================================
# DOWNLOAD DO ARQUIVO
# =====================================
try:
    response = requests.get(url_csv)
    if response.status_code != 200:
        raise Exception(f"Falha ao baixar arquivo ({response.status_code}): {url_csv}")
    
    # Escreve no diretório do volume (Databricks Volumes já montado)
    with open(destino_final, "wb") as f:
        f.write(response.content)
    
    print(f"Arquivo {destino_final} gravado com sucesso ✅")

except Exception as e:
    raise RuntimeError(f"Erro no download do arquivo INPE: {e}")

In [0]:
# =====================================
#  LEITURA PARA VERIFICAÇÃO
# =====================================
df = spark.read.option("header", True).option("delimiter", ",").csv(destino_final)
print(f"Arquivo contém {df.count()} registros.")